In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
TRN = 'data/train_ver2.csv'
TST = 'data/test_ver2.csv'

print('# Loading data..')
target_cols = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
               'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
               'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
               'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
               'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1',
               'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

# use 2015-06 data only
trn = pd.read_csv(TRN, usecols=target_cols + ['fecha_dato', 'ncodpers'])
trn_may = trn[trn['fecha_dato'] == '2015-05-28'].drop(['fecha_dato'], axis=1).set_index('ncodpers')
trn_jun = trn[trn['fecha_dato'] == '2015-06-28'].drop(['fecha_dato'], axis=1).set_index('ncodpers')
trn_may.columns = [col + '_may' for col in target_cols]
trn_jun.columns = [col + '_jun' for col in target_cols]

# calculating 'purchase' from 'possessions' by subtracting May target values from June, this is naive method
labels = trn_jun.join(trn_may)
labels.fillna(0, inplace=True)
for col in target_cols:
    labels[col] = labels[col + '_jun'] - labels[col + '_may']
labels = labels[target_cols]
index = labels[labels.sum(axis=1) > 0].index
labels = labels.loc[index]

trn = pd.read_csv(TRN)
trn = trn[trn['fecha_dato'] == '2015-06-28'].set_index('ncodpers')
trn = trn.loc[index]
trn[target_cols] = labels

del labels, index, trn_may, trn_jun

# melt 24 classes into single multi-class
trn_melt = []
for _, row in trn.iterrows():
    for i in range(24):
        if row[23 + i] == 1:
            temp = row[:23].values.tolist()
            temp.append(i)
            trn_melt.append(temp)
trn = pd.DataFrame(np.asarray(trn_melt), columns=trn.columns.tolist()[:23] + ['target'])
y = trn['target'].astype(int)

# drop targets with too few frequency
rem_targets = [23, 22, 2, 21, 18, 17, 4, 11, 12, 9, 6, 13, 7, 19, 8]
trn = trn[y.isin(rem_targets)]
y = y[y.isin(rem_targets)]
y = LabelEncoder().fit_transform(y)

tst = pd.read_csv(TST)
tst.drop(['ncodpers'], axis=1, inplace=True)
print('# trn shape : {} | y shape : {} | tst shape : {}'.format(trn.shape, y.shape, tst.shape))

# Loading data..


/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# trn shape : (38230, 24) | y shape : (38230,) | tst shape : (929615, 23)


In [4]:
trn.to_csv('data/baseline.clean.trn.csv', index=False)
tst.to_csv('data/baseline.clean.tst.csv', index=False)
pd.DataFrame(y).to_csv('data/baseline.clean.y.csv', index=False)